Importing libraries and modules

In [113]:
import urllib
from datetime import datetime
import csv
import pandas as pd

 ---

Global variables

In [114]:
PATH = "C:\\Users\\serma\\Desktop\\Універ\\2 курс\\2 семестр\\АД\\Lab1\\files\\"

regions = {
    1: 22,
    2: 24,
    3: 23,
    4: 25,
    5: 3,
    6: 4,
    7: 8,
    8: 19,
    9: 20,
    10: 21,
    11: 9,
    12: '-',
    13: 10,
    14: 11,
    15: 12,
    16: 13,
    17: 14,
    18: 15,
    19: 16,
    20: '-',
    21: 17,
    22: 18,
    23: 6,
    24: 1,
    25: 2,
    26: 7,
    27: 5,
}

---

Function for creating DataFrame from csv file

In [115]:
def obl_read(filename, i):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(filename, header = 1, names = headers) # read csv, create df
    df.drop('empty', inplace=True, axis=1) # drop empty column
    df.drop(df.tail(1).index,inplace=True) # drop last row
    df['Year'] = df['Year'].replace(['<tt><pre>1982'], '1982') # delete tags from first value
    df = df.drop(df.loc[df['VHI'] == -1].index) # drop all nan values
    df['area'] = i # service reg number
    df['area'].replace({i:regions[i]}, inplace = True) # task reg number
    return df
    

---

Main code for writing csv files and creating big df

In [91]:
for i in range(1, 28):
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2020&type=Mean'
    
    wp = urllib.request.urlopen(url)
    text = wp.read()
    
    now = datetime.now()
    date_and_time_time = now.strftime("%d%m%Y%H%M%S")
    
    with open(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", 'wb') as f:  
        f.write(text)
    
    if i == 1:
        df = obl_read(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", i)
    else:
        df = pd.concat([df, obl_read(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", i)])
    # print(obl_read(f"{PATH}NOAA_ID{i}_{date_and_time_time}.csv", i).head())
    
    
print("VHI is downloaded...")

VHI is downloaded...


---

Doing tasks

In [116]:
# get min and max vhi values for every area and year
min_grouped_df = df.groupby(['area', 'Year'])['VHI'].min()
max_grouped_df = df.groupby(['area', 'Year'])['VHI'].max()

# make it prettier
data = {'min': min_grouped_df,
        'max': max_grouped_df}
vhi_df = pd.DataFrame(data)
vhi_df

min    max
area Year              
1    1982  19.92  59.42
     1983  31.16  54.04
     1984  31.54  76.06
     1985  33.15  60.35
     1986  23.15  44.45
...          ...    ...
-    2016  39.68  67.54
     2017  36.14  68.67
     2018  36.43  59.54
     2019  24.95  59.68
     2020  30.71  60.83

[1014 rows x 2 columns]

In [117]:
# years with extreme drought
vhi_df[vhi_df['min'] < 15]

min    max
area Year              
1    2000  11.25  63.27
2    1994  11.91  62.60
4    1986  14.99  47.87
     1993   6.26  78.12
7    2007  10.88  48.77
9    2000  10.60  61.87
11   1986  12.45  48.82
     1993  13.10  85.63
13   2007   5.94  57.44
14   2007   5.52  56.82
17   1984  12.64  66.51
19   2000   9.36  68.45
20   2003  14.53  67.71
     2007  12.23  50.38
22   2000  10.68  72.05
25   2007  13.28  58.35
     2012  13.54  42.64
-    1999  12.88  70.62
     2000   6.49  71.94

In [118]:
# years with moderate drought
vhi_df[(vhi_df['min'] < 35) & (vhi_df['min'] > 15)]

min    max
area Year              
1    1982  19.92  59.42
     1983  31.16  54.04
     1984  31.54  76.06
     1985  33.15  60.35
     1986  23.15  44.45
...          ...    ...
-    2012  28.21  52.37
     2013  28.67  64.29
     2015  34.52  68.59
     2019  24.95  59.68
     2020  30.71  60.83

[696 rows x 2 columns]